In [1]:
import torch
import torchaudio
import torch.nn.functional as F

import os
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from IPython.display import Audio
import time

## 1. Creating the Dataset

In [ ]:
from torchaudio.transforms import Resample
from data import LibriDataset

In [ ]:
SAMPLE_FREQ = 16000
AUDIO_MAX_DURATION = 60
BATCH_SIZE = 12

# Creating the dataset and dataloader

trainDataset = LibriDataset("data_cut", newSampleFreq = SAMPLE_FREQ, maxLenght = SAMPLE_FREQ * AUDIO_MAX_DURATION)
trainDataLoader = DataLoader(trainDataset, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
print("Audio in the dataset: " + str(len(trainDataset)))
print("Frame per item: " + str(trainDataset[2][0].shape))
Audio(trainDataset[40][0], rate = SAMPLE_FREQ)

### 2. HuBERT TEST

In [ ]:
from transformers import HubertModel,AutoProcessor,AutoFeatureExtractor,Wav2Vec2Processor,HubertForCTC
import joblib
import torch.nn as nn
from hubertKM import SemanticTokenizer, visualizeEmbeddings

#### Importing the pretrained models for HuBERT and KMeans

In [ ]:
# A semantic tokenizer
# Input: (w2vCheckpointPath, kmeansCheckpointPath)
# Output: (semanticTokens, normalizedEmbeddings)
w2vBERT = SemanticTokenizer("facebook/hubert-base-ls960","./hubertKM/hubert_base_ls960_L9_km500.bin")  

#### Computing semantic tokens

In [ ]:
with torch.no_grad():
    s = time.time()
    ##semanticTokens, embeddings = w2vBERT(trainDataset[0])
    #semanticTokens, embeddings = w2vBERT(next(iter(trainDataLoader)).squeeze())
    print(time.time() - s)

In [ ]:
#semanticTokens, embeddings

#### Visualizing embeddings

In [ ]:
# Create some plots using PCA and t-SNE

# visualizeEmbeddings(embeddings, semanticTokens)

#### Check with another implementation (OPTIONAL)

## 3. Creating the Semantic Transformer

In [ ]:
from SoundStream import soundstream_16khz, audio_to_tokens, tokens_to_audio, encode_audio, decode_audio

In [ ]:
Audio(trainDataset[40][0], rate = SAMPLE_FREQ)

In [ ]:
soundStream = soundstream_16khz()

## TEST

In [ ]:
from data import storeTokens

fileCount = storeTokens("data_cut","out", w2vBERT, soundStream, fileCountCheckpoint = 2)
fileCount

In [ ]:
with open("out/out.csv", mode='r', newline='') as file:
    content = csv.reader(file, delimiter = ";")
    for row in content:
        print(row[0])
    #print(sum(1 for row in content) - 2, "records")

In [ ]:
for root, dirs, files in os.walk("data_cut"):
        for file in files:
            if file.endswith(".flac"):
                print(file)